In [4]:
import pandas as pd
from pathlib import Path
import json
from IPython.display import display, HTML

# Set pandas display options to avoid truncation
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

def print_section(title, char="=", width=80):
    """Print a formatted section header."""
    print(f"\n{char * width}")
    print(title.center(width))
    print(f"{char * width}\n")

def print_item(label, value, indent=2):
    """Print a labeled item with consistent formatting."""
    spaces = " " * indent
    print(f"{spaces}{label}: {value}")

def clean_csv_duplicates(folder_path, structure_coords_path, suffix="_cleaned"):
    """
    Remove duplicate structure ID and coordinate pairs from all CSVs in a folder.
    
    Args:
        folder_path: Path to folder containing CSV files to clean
        structure_coords_path: Path to structure_coordinates.csv (won't be modified)
        suffix: Suffix to append to cleaned files (default: "_cleaned")
    """
    folder = Path(folder_path)
    structure_coords_file = Path(structure_coords_path)
    
    # Get all CSV files except structure_coordinates.csv
    csv_files = [f for f in folder.glob("*.csv") 
                 if f.name != structure_coords_file.name and not f.name.endswith(f"{suffix}.csv")]
    
    if not csv_files:
        print(f"\n⚠️  No CSV files found in {folder_path}")
        return
    
    print_section(f"CSV DUPLICATE CLEANER - Processing {len(csv_files)} files")
    
    summary_data = []
    
    for csv_file in csv_files:
        print(f"\n{'─' * 80}")
        print(f"📄 FILE: {csv_file.name}")
        print(f"{'─' * 80}")
        
        # Load CSV
        df = pd.read_csv(csv_file)
        original_rows = len(df)
        
        # Get structure ID column (first column)
        id_column = df.columns[0]
        
        # Check if COORDINATES column exists
        if 'COORDINATES' not in df.columns:
            print(f"  ⚠️  No COORDINATES column found - SKIPPING\n")
            summary_data.append({
                'File': csv_file.name,
                'Status': 'Skipped - No COORDINATES',
                'Original Rows': original_rows,
                'Final Rows': original_rows,
                'Duplicates Removed': 0
            })
            continue
        
        # Report on null values
        null_ids = df[id_column].isna().sum()
        null_coords = df['COORDINATES'].isna().sum()
        
        print_item("Structure ID Column", id_column)
        print_item("Original Rows", f"{original_rows:,}")
        
        if null_ids > 0:
            print_item("Null Structure IDs", f"⚠️  {null_ids:,} rows")
        if null_coords > 0:
            print_item("Null COORDINATES", f"⚠️  {null_coords:,} rows")
        
        # Remove duplicate (structure ID, COORDINATES) pairs
        # Keep the first occurrence
        df_cleaned = df.drop_duplicates(subset=[id_column, 'COORDINATES'], keep='first')
        
        duplicates_removed = original_rows - len(df_cleaned)
        
        if duplicates_removed > 0:
            print_item("Duplicates Found", f"🔴 {duplicates_removed:,} rows")
            print_item("Action", "Removed duplicates, kept first occurrence")
        else:
            print_item("Duplicates Found", "✅ None")
        
        print_item("Final Rows", f"{len(df_cleaned):,}")
        
        # Save cleaned CSV
        output_name = csv_file.stem + suffix + csv_file.suffix
        output_path = folder / output_name
        df_cleaned.to_csv(output_path, index=False)
        print_item("Saved As", output_name)
        
        summary_data.append({
            'File': csv_file.name,
            'Status': 'Cleaned' if duplicates_removed > 0 else 'No Changes',
            'Original Rows': original_rows,
            'Final Rows': len(df_cleaned),
            'Duplicates Removed': duplicates_removed
        })
    
    # Display summary table
    print_section("SUMMARY", char="=")
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
    
    total_duplicates = summary_df['Duplicates Removed'].sum()
    print(f"\n✅ COMPLETE: Removed {total_duplicates:,} total duplicate rows across all files\n")

def clean_and_verify(folder_path, structure_coords_path, suffix="_cleaned"):
    """
    Clean CSVs and verify against structure_coordinates.csv source of truth.
    
    Args:
        folder_path: Path to folder containing CSV files to clean
        structure_coords_path: Path to structure_coordinates.csv
        suffix: Suffix to append to cleaned files
    """
    # Load structure coordinates as source of truth
    structure_coords = pd.read_csv(structure_coords_path)
    id_col_truth = structure_coords.columns[0]
    
    print_section("SOURCE OF TRUTH")
    print_item("File", Path(structure_coords_path).name)
    print_item("Total Structure IDs", f"{len(structure_coords):,}")
    print_item("Valid Coordinates", f"{structure_coords['COORDINATES'].notna().sum():,}")
    
    # First clean duplicates
    clean_csv_duplicates(folder_path, structure_coords_path, suffix)
    
    # Now verify cleaned files
    folder = Path(folder_path)
    cleaned_files = list(folder.glob(f"*{suffix}.csv"))
    
    if cleaned_files:
        print_section("VERIFICATION AGAINST SOURCE OF TRUTH", char="=")
        
        verification_data = []
        
        for csv_file in cleaned_files:
            print(f"\n{'─' * 80}")
            print(f"🔍 VERIFYING: {csv_file.name}")
            print(f"{'─' * 80}")
            
            df = pd.read_csv(csv_file)
            id_column = df.columns[0]
            
            if 'COORDINATES' in df.columns:
                # Check how many structure IDs exist in source of truth
                valid_ids = df[id_column].isin(structure_coords[id_col_truth]).sum()
                invalid_ids = len(df) - valid_ids
                
                print_item("Total Rows", f"{len(df):,}")
                print_item("IDs in Source of Truth", f"✅ {valid_ids:,} / {len(df):,}")
                
                if invalid_ids > 0:
                    print_item("IDs NOT in Source", f"⚠️  {invalid_ids:,}")
                
                # Check coordinate mismatches
                merged = df.merge(
                    structure_coords[[id_col_truth, 'COORDINATES']], 
                    left_on=id_column, 
                    right_on=id_col_truth, 
                    how='inner',
                    suffixes=('_file', '_truth')
                )
                
                coord_status = "N/A"
                mismatches = 0
                
                if len(merged) > 0:
                    mismatches = (merged['COORDINATES_file'] != merged['COORDINATES_truth']).sum()
                    if mismatches > 0:
                        coord_status = f"⚠️  {mismatches:,} mismatches"
                        print_item("Coordinate Mismatches", f"🔴 {mismatches:,} found")
                    else:
                        coord_status = "✅ All match"
                        print_item("Coordinate Verification", "✅ All coordinates match source")
                
                verification_data.append({
                    'File': csv_file.name,
                    'Total Rows': len(df),
                    'Valid IDs': valid_ids,
                    'Invalid IDs': invalid_ids,
                    'Coord Status': coord_status
                })
        
        # Display verification summary
        print_section("VERIFICATION SUMMARY", char="=")
        verification_df = pd.DataFrame(verification_data)
        display(verification_df)
        
        print("\n✅ VERIFICATION COMPLETE\n")

## Example Usage - Basic Cleaning

Remove duplicate (structure ID, COORDINATES) pairs from all CSVs:

In [5]:
# Basic cleaning - just remove duplicates
folder = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
structure_coords = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"

clean_csv_duplicates(folder, structure_coords, suffix="_cleaned")


                   CSV DUPLICATE CLEANER - Processing 8 files                   


────────────────────────────────────────────────────────────────────────────────
📄 FILE: design_maps.csv
────────────────────────────────────────────────────────────────────────────────
  Structure ID Column: STRUCTURE_ID
  Original Rows: 4,914
  Duplicates Found: ✅ None
  Final Rows: 4,914
  Saved As: design_maps_cleaned.csv

────────────────────────────────────────────────────────────────────────────────
📄 FILE: macrostrat.csv
────────────────────────────────────────────────────────────────────────────────
  Structure ID Column: STRUCTURE_ID
  Original Rows: 4,914
  Duplicates Found: ✅ None
  Final Rows: 4,914
  Saved As: macrostrat_cleaned.csv

────────────────────────────────────────────────────────────────────────────────
📄 FILE: nbi_nominal.csv
────────────────────────────────────────────────────────────────────────────────
  Structure ID Column: STRUCTURE_ID
  Original Rows: 17,328
  Duplicates F

,File,Status,Original Rows,Final Rows,Duplicates Removed
0,design_maps.csv,No Changes,4914,4914,0
1,macrostrat.csv,No Changes,4914,4914,0
2,nbi_nominal.csv,Cleaned,17328,4914,12414
3,nbi_numerical.csv,Cleaned,17328,4914,12414
4,nbi_numerical_coded.csv,Cleaned,17328,4914,12414
5,nbi_semantic_nominal_derived.csv,Skipped - No COORDINATES,4914,4914,0
6,nfhl_fema_flood.csv,Cleaned,17328,4914,12414
7,nshm_hazard_grid.csv,Skipped - No COORDINATES,4914,4914,0



✅ COMPLETE: Removed 49,656 total duplicate rows across all files



## Example Usage - Clean and Verify

Remove duplicates AND verify against structure_coordinates.csv as source of truth:

In [6]:
# Clean and verify against source of truth
folder = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
structure_coords = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"

clean_and_verify(folder, structure_coords, suffix="_cleaned")


                                SOURCE OF TRUTH                                 

  File: structure_coordinates.csv
  Total Structure IDs: 4,914
  Valid Coordinates: 4,914

                   CSV DUPLICATE CLEANER - Processing 8 files                   


────────────────────────────────────────────────────────────────────────────────
📄 FILE: design_maps.csv
────────────────────────────────────────────────────────────────────────────────
  Structure ID Column: STRUCTURE_ID
  Original Rows: 4,914
  Duplicates Found: ✅ None
  Final Rows: 4,914
  Saved As: design_maps_cleaned.csv

────────────────────────────────────────────────────────────────────────────────
📄 FILE: macrostrat.csv
────────────────────────────────────────────────────────────────────────────────
  Structure ID Column: STRUCTURE_ID
  Original Rows: 4,914
  Duplicates Found: ✅ None
  Final Rows: 4,914
  Saved As: macrostrat_cleaned.csv

────────────────────────────────────────────────────────────────────────────────
📄 FILE

,File,Status,Original Rows,Final Rows,Duplicates Removed
0,design_maps.csv,No Changes,4914,4914,0
1,macrostrat.csv,No Changes,4914,4914,0
2,nbi_nominal.csv,Cleaned,17328,4914,12414
3,nbi_numerical.csv,Cleaned,17328,4914,12414
4,nbi_numerical_coded.csv,Cleaned,17328,4914,12414
5,nbi_semantic_nominal_derived.csv,Skipped - No COORDINATES,4914,4914,0
6,nfhl_fema_flood.csv,Cleaned,17328,4914,12414
7,nshm_hazard_grid.csv,Skipped - No COORDINATES,4914,4914,0



✅ COMPLETE: Removed 49,656 total duplicate rows across all files


                      VERIFICATION AGAINST SOURCE OF TRUTH                      


────────────────────────────────────────────────────────────────────────────────
🔍 VERIFYING: design_maps_cleaned.csv
────────────────────────────────────────────────────────────────────────────────
  Total Rows: 4,914
  IDs in Source of Truth: ✅ 4,914 / 4,914
  Coordinate Mismatches: 🔴 12,414 found

────────────────────────────────────────────────────────────────────────────────
🔍 VERIFYING: macrostrat_cleaned.csv
────────────────────────────────────────────────────────────────────────────────
  Total Rows: 4,914
  IDs in Source of Truth: ✅ 4,914 / 4,914
  Coordinate Mismatches: 🔴 12,414 found

────────────────────────────────────────────────────────────────────────────────
🔍 VERIFYING: nbi_nominal_cleaned.csv
────────────────────────────────────────────────────────────────────────────────
  Total Rows: 4,914
  IDs in Source of Truth: ✅

,File,Total Rows,Valid IDs,Invalid IDs,Coord Status
0,design_maps_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"
1,macrostrat_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"
2,nbi_nominal_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"
3,nbi_numerical_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"
4,nbi_numerical_coded_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"
5,nfhl_fema_flood_cleaned.csv,4914,4914,0,"⚠️ 12,414 mismatches"



✅ VERIFICATION COMPLETE



## Simple Coordinate Append

Append COORDINATES column from structure_coordinates.csv to target CSVs, placing it at position 1 (second column). Assumes row order matches.

In [2]:
import pandas as pd
from pathlib import Path

def append_coordinates_simple(csv_path, structure_coords_path, output_path=None):
    """
    Append COORDINATES column from structure_coordinates.csv to target CSV at position 1.
    Assumes row order matches between files.
    
    Args:
        csv_path: Path to target CSV file
        structure_coords_path: Path to structure_coordinates.csv
        output_path: Optional output path (defaults to overwriting input)
    """
    if output_path is None:
        output_path = csv_path
    
    # Load files
    df = pd.read_csv(csv_path)
    coords = pd.read_csv(structure_coords_path)
    
    # Check row count match
    if len(df) != len(coords):
        print(f"⚠️  WARNING: Row count mismatch!")
        print(f"   Target CSV: {len(df):,} rows")
        print(f"   Coordinates CSV: {len(coords):,} rows")
        return None
    
    # Remove COORDINATES if it already exists
    if 'COORDINATES' in df.columns:
        df = df.drop(columns=['COORDINATES'])
        print(f"ℹ️  Removed existing COORDINATES column")
    
    # Insert COORDINATES at position 1 (second column)
    df.insert(1, 'COORDINATES', coords['COORDINATES'])
    
    # Save
    df.to_csv(output_path, index=False)
    print(f"✅ Saved: {Path(output_path).name}")
    print(f"   Columns: {list(df.columns[:3])}... ({len(df.columns)} total)")
    print(f"   Rows: {len(df):,}")
    
    return df

def append_coordinates_folder(folder_path, structure_coords_path, suffix="_coords"):
    """
    Append COORDINATES to all CSVs in a folder (except structure_coordinates.csv).
    
    Args:
        folder_path: Path to folder containing CSV files
        structure_coords_path: Path to structure_coordinates.csv
        suffix: Suffix for output files (default: "_coords")
    """
    folder = Path(folder_path)
    structure_coords_file = Path(structure_coords_path)
    
    # Get all CSV files except structure_coordinates.csv and already processed files
    csv_files = [f for f in folder.glob("*.csv") 
                 if f.name != structure_coords_file.name 
                 and not f.name.endswith(f"{suffix}.csv")]
    
    if not csv_files:
        print(f"⚠️  No CSV files found in {folder_path}")
        return
    
    print("="*80)
    print(f"APPENDING COORDINATES TO {len(csv_files)} FILES")
    print("="*80 + "\n")
    
    for idx, csv_file in enumerate(csv_files, 1):
        print(f"[{idx}/{len(csv_files)}] 📄 {csv_file.name}")
        
        # Create output path with suffix
        output_name = csv_file.stem + suffix + csv_file.suffix
        output_path = folder / output_name
        
        try:
            append_coordinates_simple(csv_file, structure_coords_path, output_path)
        except Exception as e:
            print(f"❌ Error: {e}")
        
        print()
    
    print("="*80)
    print("✅ COMPLETE")
    print("="*80)

### Example Usage

In [3]:
# Single file example
csv_file = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\nshm_hazard_grid.csv"
coords_file = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"

df = append_coordinates_simple(csv_file, coords_file)

# Show result
if df is not None:
    print(f"\nFirst 3 rows, first 4 columns:")
    display(df.iloc[:3, :4])

✅ Saved: nshm_hazard_grid.csv
   Columns: ['STRUCTURE_ID', 'COORDINATES', 'PGA_475']... (14 total)
   Rows: 4,914

First 3 rows, first 4 columns:


,STRUCTURE_ID,COORDINATES,PGA_475,PGA_975
0,,"(47.98571667, -122.2271222)",0.259826,0.364282
1,,"(47.697975, -122.6195)",0.386814,0.549207
2,,"(48.21215, -121.9331306)",0.203070,0.284565


In [ ]:
# Process entire folder
folder = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
coords_file = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\structure_coordinates.csv"

append_coordinates_folder(folder, coords_file, suffix="_coords")

## Check for Duplicate Titles in Schemas

Quick script to verify no duplicate titles exist across all schema fields.

In [1]:
import json
from pathlib import Path
from collections import Counter

def check_duplicate_titles(schema_folder):
    """Check for duplicate titles across all schema files."""
    schema_folder = Path(schema_folder)
    
    all_titles = []
    title_sources = {}  # Track which file each title comes from
    
    # Get all JSON files
    schema_files = sorted(schema_folder.glob("*.json"))
    
    print(f"Checking {len(schema_files)} schema files...\n")
    
    for schema_file in schema_files:
        with open(schema_file, 'r', encoding='utf-8') as f:
            schema = json.load(f)
        
        for field_name, field_data in schema.items():
            if isinstance(field_data, dict) and 'title' in field_data:
                title = field_data['title']
                all_titles.append(title)
                
                # Track source
                if title not in title_sources:
                    title_sources[title] = []
                title_sources[title].append((schema_file.name, field_name))
    
    # Find duplicates
    title_counts = Counter(all_titles)
    duplicates = {title: count for title, count in title_counts.items() if count > 1}
    
    print(f"📊 Total titles found: {len(all_titles)}")
    print(f"📊 Unique titles: {len(title_counts)}")
    
    if duplicates:
        print(f"\n⚠️  Found {len(duplicates)} duplicate titles:\n")
        for title, count in sorted(duplicates.items()):
            print(f"  '{title}' appears {count} times:")
            for source_file, field_name in title_sources[title]:
                print(f"    - {source_file}: {field_name}")
            print()
    else:
        print("\n✅ No duplicate titles found!")
    
    return duplicates

# Run check
schema_folder = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\final_schemas"
duplicates = check_duplicate_titles(schema_folder)

Checking 7 schema files...

📊 Total titles found: 121
📊 Unique titles: 118

⚠️  Found 3 duplicate titles:

  'Approach Guardrail Adequacy' appears 2 times:
    - nbi_nominal_schema_master.json: APPR_RAIL_036C
    - nbi_nominal_schema_master.json: APPR_RAIL_END_036D

  'Route Classification' appears 2 times:
    - nbi_nominal_schema_master.json: ROUTE_PREFIX_005B
    - nbi_nominal_schema_master.json: SERVICE_LEVEL_005C

  'Vertical Clearance' appears 2 times:
    - nbi_numerical_coded_schema_master.json: VERT_CLR_OVER_MT_053
    - nbi_numerical_coded_schema_master.json: VERT_CLR_UND_054B

